In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import networkx as nx
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LeakyReLU, ReLU
from keras.utils import np_utils,  to_categorical
from keras import optimizers

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


/kaggle/input/iupui-link-prediction/features.csv
/kaggle/input/iupui-link-prediction/train_edges.csv
/kaggle/input/iupui-link-prediction/sample_submission.csv


In [2]:
train_file = "/kaggle/input/iupui-link-prediction/train_edges.csv";
sample_sub = "/kaggle/input/iupui-link-prediction/sample_submission.csv"
feat_file = "/kaggle/input/iupui-link-prediction/features.csv";

In [3]:
def calc_index(G):
    
    s = list(G.nodes);
    num_nodes = max(s)+1;
    
    #Store it into a matrix for self preservation
    #num_nodes = G.number_of_nodes();
    common_neigh = np.zeros((num_nodes,num_nodes));
    salton = common_neigh.copy();
    jaccard = common_neigh.copy();
    sorensen = common_neigh.copy();
    hub_promoted = common_neigh.copy();
    hub_depressed = common_neigh.copy();
    leicht = common_neigh.copy();
    pref = common_neigh.copy();
    adamic = common_neigh.copy();
    resource = common_neigh.copy();
    
    for k in s:
        i = s.index(k);
        if (i % 100 == 0):
            print(i)
        for l in s:
            j = s.index(l);
            if (i != j):
                deg_i = G.degree[i];
                deg_j = G.degree[j];
                min_deg = deg_i;
                max_deg = deg_j;
                if (deg_j < min_deg):
                    min_deg = deg_j;
                    max_deg = deg_i;

                com = sorted(nx.common_neighbors(G,i,j));
                num_com = len(com);
                common_neigh[i,j] = num_com;
                preff = deg_i * deg_j;
                
                ada = 0;
                res = 0;
                
                for k in com:
                    ada += 1 / math.log(G.degree[k]);
                    res += 1 / (G.degree[k]);
                    
                pref[i,j] = preff;
                adamic[i,j] = ada;
                resource[i,j] = res;

                if (num_com > 0):
                    salt = num_com / math.sqrt(deg_i * deg_j);
                    jac = num_com / (deg_i + deg_j - num_com);
                    sor = 2 * num_com / (deg_i + deg_j);
                    hub_d = num_com / min_deg;
                    hub_p = num_com / max_deg;
                    lec = num_com / (deg_i * deg_j);

                    salton[i,j] = salt;
                    jaccard[i,j] = jac;
                    sorensen[i,j] = sor;
                    hub_promoted[i,j] = hub_p;
                    hub_depressed[i,j] = hub_d;
                    leicht[i,j] = lec;
                
    
    return(common_neigh,salton,jaccard,sorensen,hub_promoted,hub_depressed,leicht,pref,adamic,resource);

def calc_common_neigh(G):
    #Store it into a matrix for self preservation
    num_nodes = G.number_of_nodes();
    common_neigh = np.zeros((num_nodes,num_nodes));
    
    for i in range(num_nodes):
        if (i % 100 == 0):
            print(i)
        for j in range(num_nodes):
            if (i != j):
                num_com = len(sorted(nx.common_neighbors(G,i,j)));
                common_neigh[i,j] = num_com
    
    return(common_neigh);

In [4]:
#Load the input files
import csv

G = nx.Graph()

with open(train_file) as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if row[1]=='1':
            edge= row[0].split('-')
            #print(int(edge[0]),int(edge[1]))
            G.add_edge(int(edge[0]),int(edge[1]))
            
#s = sorted(G.nodes);
s = G.nodes
s = list(s)

adj = nx.adjacency_matrix(G);
print(adj[146,1356])
print(adj[s.index(146),s.index(1356)])

0
1


In [5]:
#print(s)
#print(sorted(s))
#print(type(s))
#print(list(s))

In [6]:
#Node info
print(G.number_of_nodes());
#nx.draw(G,node_size = 1);

2506


In [7]:
# next let's read in the possible edges we need to classify from the test file
#FUCK THE WAY MOHLER DID THIS
#It assumes all nodes are connected so fuck
#But we will let it stand...

Gsub = nx.Graph()

with open(sample_sub) as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if row[1]=='1' or row[1]=='0':
            edge= row[0].split('-')
            #print(edge[0],edge[1]);
            Gsub.add_edge(int(edge[0]),int(edge[1]))
            
G.add_nodes_from(Gsub.nodes)
print(G.number_of_nodes());

2708


In [8]:
#print(sorted(G.nodes))
s = list(G.nodes);
#print(s)
#a = 0;
#for i in range(len(s)-1):
#    if(s[i+1] - s[i] != 1):
#        a += 1;
#print(a)
#print(G.degree[10])

In [9]:
#n = calc_common_neigh(G)
com,salt,jac,sor,hub_p,hub_d,lec,pref,ada,res = calc_index(G);


#print(com,salt,jac,sor,hub_p,hub_d);

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700


In [10]:
#Select only the training data
train_df = pd.read_csv(train_file);
train_df = train_df.values;
sz = np.shape(train_df);
train_data = np.zeros((sz[0],3));

for i in range(sz[0]):
    edge = train_df[i,0];
    edge = edge.split('-');
    train_data[i,0] = int(edge[0])
    train_data[i,1] = int(edge[1]);
    train_data[i,2] = int(train_df[i,1]);
    
#print(train_data)

#Create a matrix of the data
data_exist = np.zeros((len(s),len(s)));
for i in range(sz[0]):
    
    j = int(train_data[i,0]);
    k = int(train_data[i,1]);
    j = s.index(j);
    k = s.index(k);
    
    data_exist[j,k] = 1;
    data_exist[k,j] = 1;

def datum_in(i,j,df):
    sz = np.shape(df);
    for k in range(sz[0]):
        if (df[k,0] == i):
            if (df[k,1] == j):
                return True;
        if (df[k,0] == j):
            if (df[k,1] == i):
                return True;
        
    return False;

def datum_in_mat(i,j,s,dat):
    
    if (data_exist[i,j] != 0):
        return True;
    else:
        return False;

In [11]:
#Load the feature file
feat = pd.read_csv(feat_file,header=None);
feat = feat.values;
print(feat)
print(feat.shape)

#feat = feat.sort(axis=0);
#print(feat)

def feat_find(x,feat):
    y = feat[feat[:,0]==x][0][1:1433];
    return y;

y = feat_find(1911,feat)
print(np.sum(y))
print(np.sum(feat[0,:]) - 1911)
y = feat_find(2002,feat)
print(np.sum(y))
print(np.sum(feat[1,:]) - 2002)

[[ 462    0    0 ...    0    0    0]
 [1911    0    0 ...    0    0    0]
 [2002    0    0 ...    0    0    0]
 ...
 [2372    0    0 ...    0    0    0]
 [ 955    0    0 ...    0    0    0]
 [ 376    0    0 ...    0    0    0]]
(2708, 1434)
17
-1429
22
-74


In [12]:
a = np.arange(3)
print(a)
print(np.sum(a))
print(np.linalg.norm(a))

[0 1 2]
3
2.23606797749979


In [13]:
#Now I need to make this into something that I can feed into a neural net like nothing
num_nodes = max(s)+1;
adj = nx.adjacency_matrix(G);
adj = adj.todense();

sz_train = np.shape(train_df);
datum = np.zeros((sz_train[0],16));
ind = 0;

for k in s:
    i = s.index(k);
    if (i % 100 == 0):
            print(i)
    for l in s:
        j = s.index(l);
        if (j > i):
            if (datum_in_mat(i,j,s,data_exist)):

                datum[ind,0] = com[i,j];
                datum[ind,1] = salt[i,j];
                datum[ind,2] = jac[i,j];
                datum[ind,3] = sor[i,j];
                datum[ind,4] = hub_p[i,j];
                datum[ind,5] = hub_d[i,j];
                datum[ind,6] = lec[i,j];
                datum[ind,7] = pref[i,j];
                datum[ind,8] = res[i,j];
                datum[ind,9] = ada[i,j];

                deg_i = G.degree[i];
                deg_j = G.degree[j];
                min_deg = deg_i;
                max_deg = deg_j;
                if (min_deg > max_deg):
                    max_deg = deg_j;
                    min_deg = deg_i;
                datum[ind,10] = min_deg;
                datum[ind,11] = max_deg;
                
                #print(ind,k,l)
                y1 = feat_find(k,feat);
                y2 = feat_find(l,feat);
                
                dp = np.dot(y1,y2);
                n1 = np.linalg.norm(y1);
                n2 = np.linalg.norm(y2);
                dp_norm = dp / (n1*n2);
                
                c_ang = math.acos(dp_norm);
                
                datum[ind,12] = dp;
                datum[ind,13] = dp_norm;
                datum[ind,14] = c_ang;

                #ind_i = s.index(i);
                #ind_j = s.index(j);

                datum[ind,-1] = adj[i,j];
                ind += 1;
        
#remove the all zero rows
#datum = datum[~np.all(datum == 0, axis=1)];
#Append an all zero row
#oof = np.zeros((1,13));
#datum = np.append(datum,oof);
#datum = datum.reshape(-1,13);
print(np.shape(datum))
print(datum)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
(7388, 16)
[[2.         0.10166571 0.01538462 ... 0.04364358 1.52713888 1.        ]
 [0.         0.         0.         ... 0.09759001 1.47305075 0.        ]
 [0.         0.         0.         ... 0.23809524 1.33039211 1.        ]
 ...
 [0.         0.         0.         ... 0.         1.57079633 0.        ]
 [0.         0.         0.         ... 0.         1.57079633 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [14]:
#print(adj)


print(adj[2250,985])
print(adj[s.index(146),s.index(1356)])
print(adj[s.index(1356),s.index(146)])

0
1
1


In [15]:
#Get test data
test_df = pd.read_csv(sample_sub);
test_df = test_df.values;
test_sz = np.shape(test_df);
test_inp = np.zeros((test_sz[0],3));

for i in range(test_sz[0]):
    edge = test_df[i,0];
    edge = edge.split('-');
    test_inp[i,0] = int(edge[0])
    test_inp[i,1] = int(edge[1]);
    test_inp[i,2] = int(test_inp[i,1]);
    
#print(train_data)

num_nodes = max(s)+1;

sz_test = np.shape(train_df);
test_datum = np.zeros((sz_test[0],15));
ind = 0;

for ind in range(test_sz[0]):

    i = int(test_inp[ind,0]);
    j = int(test_inp[ind,1]);
    k = i;
    l = j;
    i = s.index(i);
    j = s.index(j);
    
    test_datum[ind,0] = com[i,j];
    test_datum[ind,1] = salt[i,j];
    test_datum[ind,2] = jac[i,j];
    test_datum[ind,3] = sor[i,j];
    test_datum[ind,4] = hub_p[i,j];
    test_datum[ind,5] = hub_d[i,j];
    test_datum[ind,6] = lec[i,j];
    test_datum[ind,7] = pref[i,j];
    test_datum[ind,8] = res[i,j];
    test_datum[ind,9] = ada[i,j];

    deg_i = G.degree[i];
    deg_j = G.degree[j];
    min_deg = deg_i;
    max_deg = deg_j;
    if (min_deg > max_deg):
        max_deg = deg_j;
        min_deg = deg_i;
    
    test_datum[ind,10] = min_deg;
    test_datum[ind,11] = max_deg;
    
    y1 = feat_find(k,feat);
    y2 = feat_find(l,feat);

    dp = np.dot(y1,y2);
    n1 = np.linalg.norm(y1);
    n2 = np.linalg.norm(y2);
    dp_norm = dp / (n1*n2);

    c_ang = math.acos(dp_norm);

    test_datum[ind,12] = dp;
    test_datum[ind,13] = dp_norm;
    test_datum[ind,14] = c_ang;

print(test_datum)

[[0.         0.         0.         ... 1.         0.03857584 1.53221092]
 [0.         0.         0.         ... 2.         0.18731716 1.38236608]
 [0.         0.         0.         ... 2.         0.09128709 1.47938197]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [16]:
train_df = pd.DataFrame(datum);
train_df.to_csv("train.csv",header=None,index=None)
test_df = pd.DataFrame(test_datum);
test_df.to_csv("test.csv",header=None,index=None)

In [17]:
#Normalization
max_col = np.amax(datum,axis=0);
datum /= max_col;
test_datum /= max_col[:-1];

In [18]:
#I now have the training and testing data
#Separate the training data into train and test
Y = datum[:,-1];
X = datum[:,:-1];

#X = X.reshape(-1,12,1);
#test_datum = test_datum.reshape(-1,12,1);

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = .2);

In [19]:
"""
# create convolution neural network
model = Sequential()
model.add(Dense(512, input_shape=(12,1)))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Flatten())

model.add(Dense(256))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Dense(128))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Dense(64))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Dense(32))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Dense(32))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.25))

model.add(Dense(1, activation='sigmoid'))

model.summary();
"""

"\n# create convolution neural network\nmodel = Sequential()\nmodel.add(Dense(512, input_shape=(12,1)))\nmodel.add(LeakyReLU(alpha=0.05))\nmodel.add(Dropout(0.33))\n\nmodel.add(Flatten())\n\nmodel.add(Dense(256))\nmodel.add(LeakyReLU(alpha=0.05))\nmodel.add(Dropout(0.33))\n\nmodel.add(Dense(128))\nmodel.add(LeakyReLU(alpha=0.05))\nmodel.add(Dropout(0.33))\n\nmodel.add(Dense(64))\nmodel.add(LeakyReLU(alpha=0.05))\nmodel.add(Dropout(0.33))\n\nmodel.add(Dense(32))\nmodel.add(LeakyReLU(alpha=0.05))\nmodel.add(Dropout(0.33))\n\nmodel.add(Dense(32))\nmodel.add(LeakyReLU(alpha=0.05))\nmodel.add(Dropout(0.25))\n\nmodel.add(Dense(1, activation='sigmoid'))\n\nmodel.summary();\n"

In [20]:
"""
#Compile model
#sgd = optimizers.SGD(lr=.01);
model.compile(loss='MSE', 
              optimizer='adagrad', 
              metrics=['accuracy'])
"""

"\n#Compile model\n#sgd = optimizers.SGD(lr=.01);\nmodel.compile(loss='MSE', \n              optimizer='adagrad', \n              metrics=['accuracy'])\n"

In [21]:
"""
history = model.fit(X_train,y_train, shuffle=True,
          batch_size=5,epochs=250,verbose=1,
          validation_data=(X_test,y_test))
          """

'\nhistory = model.fit(X_train,y_train, shuffle=True,\n          batch_size=5,epochs=250,verbose=1,\n          validation_data=(X_test,y_test))\n          '

In [22]:
"""
Y_out = model.predict(test_datum)
print(Y_out)
"""

'\nY_out = model.predict(test_datum)\nprint(Y_out)\n'

In [23]:
#"""
#Create and train the random forest classifier
for i in [3,4,5,6,7,8]:
    rf_model = RandomForestClassifier(max_depth=i,n_estimators=2500);
    rf_model.fit(X_train,y_train);
    Y_pred = rf_model.predict(X_test);
    a = accuracy_score(y_test,Y_pred);
    print(i,a);
#"""

3 0.7334235453315291
4 0.7374830852503383
5 0.7395128552097429
6 0.7395128552097429
7 0.7395128552097429
8 0.7354533152909337


In [24]:
#We choose XXXX RF since best accuracy
#print(np.mean(Y_pred))
#print(np.mean(y_test))
#"""
#We choose max_depth of 5 as our model
rf_model = RandomForestClassifier(max_depth=5,n_estimators=2500);
rf_model.fit(datum[:,:-1],datum[:,-1]);
Y_pred = rf_model.predict(X_test);
a = accuracy_score(y_test,Y_pred);
print(i,a);

Y_out = rf_model.predict(test_datum)
print(Y_out)
print(np.mean(Y_out))
#"""

8 0.7408660351826793
[0. 1. 0. ... 0. 0. 0.]
0.18854899837574446


In [25]:
#"""
# here is a simple algorithm to get you started...for each possible edge i-j in the test set
# we will count the number of nodes in the training graph they have in common...if they have
# one or more "mutual friends" then we will connect them (this will score about .67 acc on kaggle)

alfa = 0;
with open('/kaggle/working/submission.csv', 'w') as csvfile:
    fieldnames = ['edge', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    with open(sample_sub) as csvfile2:
        reader = csv.reader(csvfile2, delimiter=',')
        for row in reader:
            if row[1]=='1' or row[1]=='0':
                edge= row[0].split('-')

                i=int(edge[0])
                j=int(edge[1])
                
                # here networkx has a common_neighbors function
                num_com_neigh=len(sorted(nx.common_neighbors(G, i, j)))
                y=0
                #if num_com_neigh>0:
                #    y=1

                edg_out=str(i)+"-"+str(j) 
                y = Y_out[alfa];
                alfa += 1;
                writer.writerow({'edge': edg_out, 'label': y})
                #print(edg_out, num_com_neigh, y)
#"""

In [26]:
"""
alfa = 0;
with open('/kaggle/working/submission_alt.csv', 'w') as csvfile:
    fieldnames = ['edge', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    with open(sample_sub) as csvfile2:
        reader = csv.reader(csvfile2, delimiter=',')
        for row in reader:
            if row[1]=='1' or row[1]=='0':
                edge= row[0].split('-')

                i=int(edge[0])
                j=int(edge[1])
                
                # here networkx has a common_neighbors function
                num_com_neigh=len(sorted(nx.common_neighbors(G, i, j)))
                y=0
                #if num_com_neigh>0:
                #    y=1

                edg_out=str(i)+"-"+str(j) 
                if (Y_out[alfa] > .5):
                    y = 1;
                alfa += 1;
                writer.writerow({'edge': edg_out, 'label': y})
                #print(edg_out, num_com_neigh, y)

"""

'\nalfa = 0;\nwith open(\'/kaggle/working/submission_alt.csv\', \'w\') as csvfile:\n    fieldnames = [\'edge\', \'label\']\n    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)\n    writer.writeheader()\n\n    with open(sample_sub) as csvfile2:\n        reader = csv.reader(csvfile2, delimiter=\',\')\n        for row in reader:\n            if row[1]==\'1\' or row[1]==\'0\':\n                edge= row[0].split(\'-\')\n\n                i=int(edge[0])\n                j=int(edge[1])\n                \n                # here networkx has a common_neighbors function\n                num_com_neigh=len(sorted(nx.common_neighbors(G, i, j)))\n                y=0\n                #if num_com_neigh>0:\n                #    y=1\n\n                edg_out=str(i)+"-"+str(j) \n                if (Y_out[alfa] > .5):\n                    y = 1;\n                alfa += 1;\n                writer.writerow({\'edge\': edg_out, \'label\': y})\n                #print(edg_out, num_com_neigh, y)\n\n'